# Brief Intro to Google ADK

This notebook shows a quick intro to Google ADK.

General steps:
1. Setup environment (use local .env and .venv)
2. Check that environment is working (including connection to Neo4j)
3. Define tool(s)
4. Define agent(s)
5. Demonstrate interaction

## Setup

In [1]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [27]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nEnvironment configured.")

ModelResponse(id='chatcmpl-BcXto5UyTbKFZsTWwnJQJQlbIW3uD', created=1748526472, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_76544d79cb', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready. How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

Environment configured.


In [4]:
# Check connection to Neo4j by sending a query
from neo4j_for_adk import graphdb

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

{'status': 'success', 'query_result': [{'message': 'Neo4j is Ready!'}]}


In [5]:
# Define a basic tool -- send a parameterized cypher query
def say_hello(from_name: str) -> dict:
    """Sends a welcome message to Neo4j from a named person. 

    Args:
        name (str): the name of the person saying hello

    Returns:
        dict: A dictionary containing the results of the query.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'query_result' key with an array of result rows.
              If 'error', includes an 'error_message' key.
    """
    return graphdb.send_query("RETURN 'Hello to you, ' + $from_name AS reply",
    {
        "from_name": from_name
    })

# Example tool usage (optional test)
print(say_hello("ABK"))


{'status': 'success', 'query_result': [{'reply': 'Hello to you, ABK'}]}


---

**2\. Define the Agent (`cypher_agent`)**

Create the **Agent** itself. An `Agent` in ADK orchestrates the interaction between the user, the LLM, and the available tools.

We configure it with several key parameters:

* `name`: A unique identifier for this agent (e.g., "cypher\_agent\_v1").  
* `model`: Specifies which LLM to use. We'll start with the `llm` variable we defined above.  
* `description`: A concise summary of the agent's overall purpose. This becomes crucial later when other agents need to decide whether to delegate tasks to *this* agent.  
* `instruction`: Detailed guidance for the LLM on how to behave, its persona, its goals, and specifically *how and when* to utilize its assigned `tools`.  
* `tools`: A list containing the actual Python tool functions the agent is allowed to use (e.g., `[hello_cypher]`).

**Best Practice:** Provide clear and specific `instruction` prompts. The more detailed the instructions, the better the LLM can understand its role and how to use its tools effectively. Be explicit about error handling if needed.

**Best Practice:** Choose descriptive `name` and `description` values. These are used internally by ADK and are vital for features like automatic delegation (covered later).

In [6]:
# Define the Cypher Agent

friendly_cypher_agent = Agent(
    name="friendly_cypher_agent_v1",
    model=llm, # defined earlier in a variable
    description="Has friendly chats with Neo4j through Cypher.",
    instruction="""You are a helpful Neo4j assistant, helping a user chat with a graph by using Cypher. 
                Be polite and friendly, introducing yourself and asking who the user is. 
                If the user provides their name, use the 'hello_cypher' tool
                to get a greeting from Neo4j on their behalf.
                If the tool returns an error, inform the user politely. 
                If the tool is successful, present the reply from Neo4j.
                """,
    tools=[say_hello], # Pass the function directly
)

print(f"Agent '{friendly_cypher_agent.name}' created.")

Agent 'friendly_cypher_agent_v1' created.


---

**3\. Setup Runner and Session Service**

To manage conversations and execute the agent, we need two more components:

* `SessionService`: Responsible for managing conversation history and state for different users and sessions. The `InMemorySessionService` is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged. We'll explore state persistence more in Step 4\.  
* `Runner`: The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the `SessionService`, and yields events representing the progress of the interaction.

In [7]:
# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "hello_cypher_app"
USER_ID = "user_abk"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=friendly_cypher_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='hello_cypher_app', User='user_abk', Session='session_001'
Runner created for agent 'friendly_cypher_agent_v1'.


---

**4\. Interact with the Agent**

We need a way to send messages to our agent and receive its responses. Since LLM calls and tool executions can take time, ADK's `Runner` operates asynchronously.

We'll define an `async` helper function (`call_agent_async`) that:

1. Takes a user query string.  
2. Packages it into the ADK `Content` format.  
3. Calls `runner.run_async`, providing the user/session context and the new message.  
4. Iterates through the **Events** yielded by the runner. Events represent steps in the agent's execution (e.g., tool call requested, tool result received, intermediate LLM thought, final response).  
5. Identifies and prints the **final response** event using `event.is_final_response()`.

**Why `async`?** Interactions with LLMs and potentially tools (like external APIs) are I/O-bound operations. Using `asyncio` allows the program to handle these operations efficiently without blocking execution.

In [8]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

---

**5\. Run the Conversation**

Let's test our setup by sending a few queries to the agent. We wrap our `async` calls in a main `async` function and run it using `await`.

Watch the output:

* See the user queries.  
* Notice the `--- Tool: say_hello called... ---` logs when the agent uses the tool.  
* Observe the agent's final responses, including how it handles the case when a user name is not provided.

In [9]:
# @title Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("Hello I'm ABK",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("Hello I'm Emil",
                                    runner=runner,
                                    user_id=USER_ID,
                                    session_id=SESSION_ID)

    await call_agent_async("I am excited",
                                    runner=runner,
                                    user_id=USER_ID,
                                    session_id=SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: Hello I'm ABK
<<< Agent Response: Neo4j says: "Hello to you, ABK!" How can I assist you further with Neo4j today?

>>> User Query: Hello I'm Emil
<<< Agent Response: Neo4j says: "Hello to you, Emil!" How can I assist you with Neo4j today?

>>> User Query: I am excited
<<< Agent Response: That's wonderful to hear, Emil! Neo4j is a fantastic tool to explore and interact with graphs. Is there anything specific you would like to explore or learn about today?


---

Congratulations\! You've successfully built and interacted with your first ADK agent. It understands the user's request, uses a tool to find information, and responds appropriately based on the tool's result.

Next, we'll build a simple multi-agent system.

## A Simple Multi-Agent Team \- Delegation for Greetings & Farewells

Using multiple agents is a common pattern in real-world applications. It allows for better modularity, specialization, and scalability. Here's how it works:

1. Creating multiple, **specialized agents**, each designed for a specific capability (e.g., one for greetings, one for querying a knowledge graph, another for GraphRAG retrieval).  
2. Designating a **root agent** (or orchestrator or coordinator) that receives the initial user request.  
3. Enabling the root agent to **delegate** the request to the most appropriate specialized sub-agent based on the user's intent.

**Why build an Agent Team?**

* **Modularity:** Easier to develop, test, and maintain individual agents.  
* **Specialization:** Each agent can be fine-tuned (instructions, model choice) for its specific task.  
* **Scalability:** Simpler to add new capabilities by adding new agents.  
* **Efficiency:** Allows using potentially simpler/cheaper models for simpler tasks (like greetings).

**To illustrate this, we will:**

1. Define another simple tool for handling greetings (`say_hello`) and farewells (`say_goodbye`).  
2. Create two new specialized sub-agents: `greeting_agent` and `farewell_agent`.  
3. Update our main weather agent (`weather_agent_v2`) to act as the **root agent**.  
4. Configure the root agent with its sub-agents, enabling **automatic delegation**.  
5. Test the delegation flow by sending different types of requests to the root agent.

---

**1\. Define Tools for Sub-Agents**

First, let's create the simple Python functions that will serve as tools for our new specialist agents. Remember, clear docstrings are vital for the agents that will use them.

In [10]:
# Define the new goodbye tool

# Ensure 'say_hello' from earlier is available if running this step independently.
# def say_hello(from_name: str) -> dict: ... 

def say_goodbye() -> str:
    """Provides a simple farewell message to conclude the conversation."""
    return graphdb.send_query("RETURN 'Goodbye from Cypher!'")

# Optional self-test
print(say_hello("Alice"))
print(say_goodbye())

{'status': 'success', 'query_result': [{'reply': 'Hello to you, Alice'}]}
{'status': 'success', 'query_result': [{"'Goodbye from Cypher!'": 'Goodbye from Cypher!'}]}


---

**2\. Define the Sub-Agents (Greeting & Farewell)**

Now, create the `Agent` instances for our specialists. Notice their highly focused `instruction` and, critically, their clear `description`. The `description` is the primary information the *root agent* uses to decide *when* to delegate to these sub-agents.

We can even use different LLMs for these sub-agents\! Let's assign GPT-4o to the Greeting Agent and keep the Farewell Agent using GPT-4o as well (you could easily switch one to Claude or Gemini if desired and API keys are set).

**Best Practice:** Sub-agent `description` fields should accurately and concisely summarize their specific capability. This is crucial for effective automatic delegation.

**Best Practice:** Sub-agent `instruction` fields should be tailored to their limited scope, telling them exactly what to do and *what not* to do (e.g., "Your *only* task is...").

In [11]:
# Define Greeting and Farewell Sub-Agents

# --- Greeting Agent ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=llm,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                    "Use the 'say_hello' tool to generate the greeting. "
                    "If the user provides their name, make sure to pass it to the tool. "
                    "Do not engage in any other conversation or tasks.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' created.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Error: {e}")

# --- Farewell Agent ---
farewell_agent = None
try:
    farewell_agent = Agent(
        # Can use the same or a different model
        model=llm, # Sticking with GPT for this example
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                    "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                    "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                    "Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' created.")
except Exception as e:
    print(f"❌ Could not create Farewell agent. Error: {e}")

✅ Agent 'greeting_agent' created.
✅ Agent 'farewell_agent' created.


---

**3\. Define the Root Agent (Cypher Agent v2) with Sub-Agents**

Our `cypher_agent` is now a root agent with sub-agents. The key changes are:

* Adding the `sub_agents` parameter: We pass a list containing the `greeting_agent` and `farewell_agent` instances we just created.  
* Updating the `instruction`: We explicitly tell the root agent *about* its sub-agents and *when* it should delegate tasks to them.

**Key Concept: Automatic Delegation (Auto Flow)** By providing the `sub_agents` list, ADK enables automatic delegation. When the root agent receives a user query, its LLM considers not only its own instructions and tools but also the `description` of each sub-agent. If the LLM determines that a query aligns better with a sub-agent's described capability (e.g., "Handles simple greetings"), it will automatically generate a special internal action to *transfer control* to that sub-agent for that turn. The sub-agent then processes the query using its own model, instructions, and tools.

**Best Practice:** Ensure the root agent's instructions clearly guide its delegation decisions. Mention the sub-agents by name and describe the conditions under which delegation should occur.

In [12]:
# @title Define the Root Agent with Sub-Agents

# Ensure sub-agents were created successfully before defining the root agent.
root_agent = None
runner_root = None # Initialize runner

cypher_agent_team = Agent(
    name="cypher_agent_v2", # Give it a new version name
    model=llm,
    description="The main coordinator agent. Handles general questions about Neo4j and delegates greetings/farewells to specialists.",
    instruction="""You are the main Cypher Agent coordinating a team. Your primary responsibility is to provide information about Neo4j. 

                You have specialized sub-agents: 
                1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. 
                2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. 
                Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. 
                If it's a question about Neo4j, handle it yourself using your knowledge. 
                For anything else, respond appropriately or state you cannot handle it.
                """,
    tools=[], # No tools for the root agent
    # Key change: Link the sub-agents here!
    sub_agents=[greeting_agent, farewell_agent]
)
root_agent = cypher_agent_team
print(f"✅ Root Agent '{cypher_agent_team.name}' created with sub-agents: {[sa.name for sa in cypher_agent_team.sub_agents]}")


✅ Root Agent 'cypher_agent_v2' created with sub-agents: ['greeting_agent', 'farewell_agent']


---

**4\. Interact with the Agent Team**

Now that we've defined our root agent (`cypher_agent_team` - *Note: Ensure this variable name matches the one defined in the previous code block, likely `# @title Define the Root Agent with Sub-Agents`, which might have named it `root_agent`*) with its specialized sub-agents, let's test the delegation mechanism.

The following code block will:

1.  Define an `async` function `run_team_conversation`.
2.  Inside this function, create a *new, dedicated* `InMemorySessionService` and a specific session (`session_001_agent_team`) just for this test run. This isolates the conversation history for testing the team dynamics.
3.  Create a `Runner` (`runner_agent_team`) configured to use our `cypher_agent_team` (the root agent) and the dedicated session service.
4.  Use our updated `call_agent_async` function to send different types of queries (greeting, farewell) to the `runner_agent_team`. We explicitly pass the runner, user ID, and session ID for this specific test.
5.  Immediately execute the `run_team_conversation` function.

We expect the following flow:

1.  The "Hello there!" query goes to `runner_agent_team`.
2.  The root agent (`cypher_agent_team`) receives it and, based on its instructions and the `greeting_agent`'s description, delegates the task.
3.  `greeting_agent` handles the query, calls its `say_hello` tool, and generates the response.
4.  The "What is Neo4j?" query is *not* delegated and is handled directly by the root agent using its public knowledge.
5.  The "Thanks, bye!" query is delegated to the `farewell_agent`, which uses its `say_goodbye` tool.



In [13]:
# @title Interact with the Agent Team

# Ensure the root agent (e.g., 'weather_agent_team' or 'root_agent' from the previous cell) is defined.
# Ensure the call_agent_async function is defined.

# Check if the root agent variable exists before defining the conversation function
root_agent_var_name = 'root_agent' 
if 'cypher_agent_team' in globals(): # Check if user used this name instead
    root_agent_var_name = 'cypher_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'weather_agent_team') not found. Cannot define run_team_conversation.")
    # Assign a dummy value to prevent NameError later if the code block runs anyway
    root_agent = None

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- Testing Agent Team Delegation ---")
        # InMemorySessionService is simple, non-persistent storage for this tutorial.
        session_service = InMemorySessionService()

        # Define constants for identifying the interaction context
        APP_NAME = "cypher_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team" # Using a fixed ID for simplicity

        # Create the specific session where the conversation will happen
        session = session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        # --- Get the actual root agent object ---
        # Use the determined variable name
        actual_root_agent = globals()[root_agent_var_name]

        # Create a runner specific to this agent team test
        runner_agent_team = Runner(
            agent=actual_root_agent, # Use the root agent object
            app_name=APP_NAME,       # Use the specific app name
            session_service=session_service # Use the specific session service
            )
        # Corrected print statement to show the actual root agent's name
        print(f"Runner created for agent '{actual_root_agent.name}'.")

        # Always interact via the root agent's runner, passing the correct IDs
        await call_agent_async(query = "Hello, I am ABK!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "What is Neo4j?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Thanks, bye!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # Execute the conversation
    # Note: This may require API keys for the models used by root and sub-agents!
    await run_team_conversation()
else:
    print("\n⚠️ Skipping agent team conversation as the root agent was not successfully defined in the previous step.")



--- Testing Agent Team Delegation ---
Session created: App='cypher_agent_team', User='user_1_agent_team', Session='session_001_agent_team'
Runner created for agent 'cypher_agent_v2'.

>>> User Query: Hello, I am ABK!
<<< Agent Response: Hello to you, ABK!

>>> User Query: What is Neo4j?
<<< Agent Response: Neo4j is a graph database management system developed by Neo4j, Inc. It's designed to store, manage, and query data in a graph structure, where data entities (nodes) and their relationships (edges) are the primary focus. This design is particularly effective for certain types of applications, such as social networks, recommendation systems, and supply chain management, because it allows for rapid queries of complex hierarchical and networked data without requiring complex join operations.

Key features of Neo4j include:

1. **Graph Model**: Neo4j uses a property-graph model that includes nodes, relationships, properties, and labels, allowing complex schemas and relationships to be m

---

Look closely at the output logs, especially the `--- Tool: ... called ---` messages. You should observe:

*   For "Hello there!", the `say_hello` tool was called (indicating `greeting_agent` handled it).
*   For "What is Neo4j?",  the root agent handled it directly.
*   For "Thanks, bye!", the `say_goodbye` tool was called (indicating `farewell_agent` handled it).

This confirms successful **automatic delegation**! The root agent, guided by its instructions and the `description`s of its `sub_agents`, correctly routed user requests to the appropriate specialist agent within the team.

You've now structured your application with multiple collaborating agents. This modular design is fundamental for building more complex and capable agent systems. In the next step, we'll give our agents the ability to remember information across turns using session state.

## Step 4: Adding Memory and Personalization with Session State

So far, our agent team can handle different tasks through delegation, but each interaction starts fresh – the agents have no memory of past conversations or user preferences within a session. To create more sophisticated and context-aware experiences, agents need **memory**. ADK provides this through **Session State**.

**What is Session State?**

* It's a Python dictionary (`session.state`) tied to a specific user session (identified by `APP_NAME`, `USER_ID`, `SESSION_ID`).  
* It persists information *across multiple conversational turns* within that session.  
* Agents and Tools can read from and write to this state, allowing them to remember details, adapt behavior, and personalize responses.

**How Agents Interact with State:**

1. **`ToolContext` (Primary Method):** Tools can accept a `ToolContext` object (automatically provided by ADK if declared as the last argument). This object gives direct access to the session state via `tool_context.state`, allowing tools to read preferences or save results *during* execution.  
2. **`output_key` (Auto-Save Agent Response):** An `Agent` can be configured with an `output_key="your_key"`. ADK will then automatically save the agent's final textual response for a turn into `session.state["your_key"]`.

**In this step, we will enhance our Cypher team by:**

1. Using a **new** `InMemorySessionService` to demonstrate state in isolation.  
2. Initializing session state with a default user name.  
3. Creating a state-aware version of the say_hello tool (`say_hello_stateful`) which responds with a personalized greeting and sets the user name in the session state.  
4. Creating a state-aware version of the say_goodbye tool (`say_goodbye_stateful`) that reads this preference via `ToolContext` and adjusts a personalized goodbye message.  
5. Running a conversation to observe how the initial state affects the tool, how manual state changes alter subsequent behavior.

---

**1\. Initialize New Session Service and State**

To clearly demonstrate state management without interference from prior steps, we'll instantiate a new `InMemorySessionService`. We'll also create a session with an initial state defining the user's preferred temperature unit.

In [14]:
# @title 1. Initialize New Session Service and State

# Import necessary session components
from google.adk.sessions import InMemorySessionService

# Create a NEW session service instance for this state demonstration
session_service_stateful = InMemorySessionService()
print("✅ New InMemorySessionService created for state demonstration.")

# Define a NEW session ID for this part of the tutorial
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# Define initial state data - user prefers Celsius initially
initial_state = {
    "user_name": "Graph Learner"
}

# Create the session, providing the initial state
session_stateful = session_service_stateful.create_session(
    app_name=APP_NAME, # Use the consistent app name
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< Initialize state during creation
)
print(f"✅ Session '{SESSION_ID_STATEFUL}' created for user '{USER_ID_STATEFUL}'.")

# Verify the initial state was set correctly
retrieved_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- Initial Session State ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("Error: Could not retrieve session.")

✅ New InMemorySessionService created for state demonstration.
✅ Session 'session_state_demo_001' created for user 'user_state_demo'.

--- Initial Session State ---
{'user_name': 'Graph Learner'}


---

**2\. Create State-Aware hello/goodbye Tools (`say_hello_stateful` and `say_goodbye_stateful`)**

Now, we create a new version of the hello/goodbye tools. The key feature is accepting `tool_context: ToolContext` which allows them to access `tool_context.state`. They will write to or read from the `user_name` state variable.


* **Key Concept: `ToolContext`** This object is the bridge allowing your tool logic to interact with the session's context, including reading and writing state variables. ADK injects it automatically if defined as the last parameter of your tool function.


* **Best Practice:** When reading from state, use `dictionary.get('key', default_value)` to handle cases where the key might not exist yet, ensuring your tool doesn't crash.

In [15]:
from google.adk.tools.tool_context import ToolContext

def say_hello_stateful(user_name: str, tool_context:ToolContext):
    """Says hello to the user, recording their name into state."""
    tool_context.state["user_name"] = user_name
    return graphdb.send_query(
        f"RETURN 'Hello to you, ' + $user_name + '.' AS reply",
    {
        "user_name": user_name
    })
    
def say_goodbye_stateful(tool_context: ToolContext) -> dict:
    """Says goodbye to the user, reading their name from state."""
    user_name = tool_context.state.get("user_name", "stranger")
    return graphdb.send_query("RETURN 'Goodbye, ' + $user_name + ', nice to chat with you!' AS reply",
    {
        "user_name": user_name
    })


print("✅ State-aware 'say_hello_stateful' and 'say_goodbye_stateful' tools defined.")


✅ State-aware 'say_hello_stateful' and 'say_goodbye_stateful' tools defined.


---

**3\. Redefine Sub-Agents and Update Root Agent**

To ensure this step is self-contained and builds correctly, we first redefine the `greeting_agent` and `farewell_agent` exactly as they were in Step 3\. Then, we define our new root agent (`cypher_agent_v4_stateful`):

* It uses the new `say_hello_stateful` and `say_goodbye_stateful` tools.  
* It includes the greeting and farewell sub-agents for delegation.  


In [16]:
# @title 3. Redefine Sub-Agents and Update Root Agent with output_key

from google.adk.agents import Agent
from google.adk.runners import Runner

# --- Redefine Greeting Agent (from Step 3) ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=llm,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello_stateful],
    )
    print(f"✅ Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Error: {e}")

# --- Redefine Farewell Agent (from Step 3) ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=llm,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye_stateful' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye_stateful' tool.",
        tools=[say_goodbye_stateful],
    )
    print(f"✅ Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Error: {e}")

# --- Define the Updated Root Agent ---
root_agent_stateful = None
runner_root_stateful = None # Initialize runner

# Check prerequisites before creating the root agent
if 'greeting_agent' and 'farewell_agent' and 'say_hello_stateful' and 'say_goodbye_stateful' in globals():

    root_agent_stateful = Agent(
        name="cypher_agent_v4_stateful", # New version name
        model=llm,
        description="Main agent: Provides weather (state-aware unit), delegates greetings/farewells, saves report to state.",
        instruction="""You are the main conversational Cypher Agent. Your job is to answer questions about Neo4j. 

                    Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. 
                    Handle only questions about Neo4j, greetings, and farewells.""",
        tools=[], # Still no tools for root
        sub_agents=[greeting_agent, farewell_agent], # Include sub-agents
    )
    print(f"✅ Root Agent '{root_agent_stateful.name}' created using agents with stateful tools.")

    # --- Create Runner for this Root Agent & NEW Session Service ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # Use the NEW stateful session service
    )
    print(f"✅ Runner created for stateful root agent '{runner_root_stateful.agent.name}' using stateful session service.")

else:
    print("❌ Cannot create stateful root agent. Prerequisites missing.")
    if not greeting_agent: print(" - greeting_agent definition missing.")
    if not farewell_agent: print(" - farewell_agent definition missing.")


✅ Agent 'greeting_agent' redefined.
✅ Agent 'farewell_agent' redefined.
✅ Root Agent 'cypher_agent_v4_stateful' created using agents with stateful tools.
✅ Runner created for stateful root agent 'cypher_agent_v4_stateful' using stateful session service.


---

**4\. Interact and Test State Flow**

Now, let's execute a conversation designed to test the state interactions using the `runner_root_stateful` (associated with our stateful agent and the `session_service_stateful`). We'll use the `call_agent_async` function defined earlier, ensuring we pass the correct runner, user ID (`USER_ID_STATEFUL`), and session ID (`SESSION_ID_STATEFUL`).

The conversation flow will be:

1.  **Check weather (London):** The `get_weather_stateful` tool should read the initial "Celsius" preference from the session state initialized in Section 1. The root agent's final response (the weather report in Celsius) should get saved to `state['last_weather_report']` via the `output_key` configuration.
2.  **Manually update state:** We will *directly modify* the state stored within the `InMemorySessionService` instance (`session_service_stateful`).
    *   **Why direct modification?** The `session_service.get_session()` method returns a *copy* of the session. Modifying that copy wouldn't affect the state used in subsequent agent runs. For this testing scenario with `InMemorySessionService`, we access the internal `sessions` dictionary to change the *actual* stored state value for `user_preference_temperature_unit` to "Fahrenheit". *Note: In real applications, state changes are typically triggered by tools or agent logic returning `EventActions(state_delta=...)`, not direct manual updates.*
3.  **Check weather again (New York):** The `get_weather_stateful` tool should now read the updated "Fahrenheit" preference from the state and convert the temperature accordingly. The root agent's *new* response (weather in Fahrenheit) will overwrite the previous value in `state['last_weather_report']` due to the `output_key`.
4.  **Greet the agent:** Verify that delegation to the `greeting_agent` still works correctly alongside the stateful operations. This interaction will become the *last* response saved by `output_key` in this specific sequence.
5.  **Inspect final state:** After the conversation, we retrieve the session one last time (getting a copy) and print its state to confirm the `user_preference_temperature_unit` is indeed "Fahrenheit", observe the final value saved by `output_key` (which will be the greeting in this run), and see the `last_city_checked_stateful` value written by the tool.


In [17]:
# @title 4. Interact to Test State Flow and output_key

# Ensure the stateful runner (runner_root_stateful) is available from the previous cell
# Ensure call_agent_async, USER_ID_STATEFUL, SESSION_ID_STATEFUL, APP_NAME are defined

if 'runner_root_stateful' in globals() and runner_root_stateful:
  async def run_stateful_conversation():
      print("\n--- Testing State: Stateful Greeting and Farewell ---")

      # 1. Say hello, introducing yourself
      print("--- Turn 1: Say 'hello'  ---")
      await call_agent_async(query= "Hello, I'm ABK",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

      # 2. Test farewell reading from state
      print("\n--- Turn 2: Say goodbye, expecting the response to be personalized ---")
      await call_agent_async(query= "Bye!",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

  # Execute the conversation
  await run_stateful_conversation()

  # Inspect final session state after the conversation
  print("\n--- Inspecting Final Session State ---")
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id= USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print(f"Final Username: {final_session.state.get('user_name')}")
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping state test conversation. Stateful root agent runner ('runner_root_stateful') is not available.")


--- Testing State: Stateful Greeting and Farewell ---
--- Turn 1: Say 'hello'  ---

>>> User Query: Hello, I'm ABK
<<< Agent Response: Hello to you, ABK.

--- Turn 2: Say goodbye, expecting the response to be personalized ---

>>> User Query: Bye!
<<< Agent Response: Goodbye, ABK, nice to chat with you!

--- Inspecting Final Session State ---
Final Username: ABK


---

By reviewing the conversation flow and the final session state printout, you can confirm:

*   **State Update:** The `say_hello_stateful` tool successfully updated `user_name` in state.
*   **State Read:** The `say_goodbye_stateful` tool correctly read `user_name` from state.
*   **Delegation:** The delegation to the `greeting_agent` for "Hi!" functioned correctly even after state modifications.
*   **Final State:** The final check confirms the user_name persisted as "ABK".

You've now successfully integrated session state to personalize agent behavior using `ToolContext`, manually manipulated state for testing `InMemorySessionService`, and observed the behavior. 

This foundational understanding of state management is key as we proceed to implementing a stateful workflow..

---

---
## Finally, An Interactive Conversation

Now, let's make this interactive so you can ask your own questions! Run the cell below. It will prompt you to enter your queries directly.

In [18]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await call_agent_async(user_query,
                                            runner=runner_root_stateful,
                                            user_id=USER_ID_STATEFUL,
                                            session_id=SESSION_ID_STATEFUL)
        print(f"Response: {response}")

# Execute the interactive conversation
await run_interactive_conversation()


>>> User Query: hello I am bob
<<< Agent Response: Hello to you, bob.
Response: None

>>> User Query: I am jerry
<<< Agent Response: Hello to you, jerry.
Response: None
